# CO Column Density Grid

November 22, 2023

Making a grid of CO 3-2/1-0 line ratios using Radex. Based on Lee's instruction from October.
Goal is to eventually use this grid to fit observed line ratios using Radex.

Ok so we can't use the `radex.run_grid` function because, as demonstrated below, it uses the deprecated `pandas.DataFrame.append` method and I don't want to go in and fix that. I might as well write my own loop wrapper; I can even try using Pool eventually.

# Column density grid with `multiprocessing`

I will do the same task as above but using the `Pool` class in `multiprocessing`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
from astropy.io import fits
from spectralradex import radex

import os
import itertools
import time
import datetime
from multiprocessing import Pool

In [ ]:
radex.get_default_parameters()

In [ ]:
# Convenience functions; copied and adapted from co_line_ratio_density.ipynb

def convert_nh2_to_co_isotope(nh2, isotope):
    # convert N(H2) to N(13CO) or N(C18O)
    n12co = nh2 * 8.5e-5 # Tielens book number
    if isotope == 'co':
        return n12co
    elif isotope == '13co':
        n13co = n12co / 44.65 # Karim et al. number
        return n13co
    elif isotope == 'c18o':
        nc18o = n12co / 417 # Wilson and Rood 1994
        return nc18o
    else:
        raise RuntimeError(f"isotope? {isotope}")


def from_multi_df_result_get_col(rdf_dict, line, isotope, colname):
    """
    Generalized function for getting a column from the rdf_dict.
    Expect CO 2-1 to be in the DataFrame as iloc[1], and index around it accordingly.
    """
    if line == 10:
        i = 0
    elif line == 32:
        i = 2
    else:
        return "Unable to get result"
    return rdf_dict[isotope].iloc[i][colname]



def from_multi_df_result_get_tr(rdf_dict, line, isotope):
    """
    Get the T_R for the given isotope of the given line from the Radex result DataFrame
    :param rdf_dict: isotope-keyed dictionary of Radex results, which are DataFrames whose rows are transitions
    :param line: int 10 or 32 indicating the transition, 1-0 or 3-2. No other options enabled right now.
    :param isotope: string indicating isotope. 'co' means 12co, '13co', and 'c18o' are self explanatory.
    :returns: float T_R (K) for the given isotope and line.
    """
    return from_multi_df_result_get_col(rdf_dict, line, isotope, "T_R (K)")


def from_multi_df_result_get_line_ratio(rdf_dict, isotope10):
    """
    3-2 / 1-0 line T_R ratio of 3-2 to 1-0
    Select the isotope of the 1-0 ratio with isotope10 arg.
    :param rdf_dict: dict of rdfs with isotope keys
    :param isotope10: '13co' or 'c18o'.
        Shouldn't use 'co' since it's optically thick;
        function will succeed, i.e. "fail silently"
    :returns: float line ratio
    """
    tr_10 = from_multi_df_result_get_tr(rdf_dict, 10, isotope10)
    tr_32 = from_multi_df_result_get_tr(rdf_dict, 32, '13co')
    return tr_32 / tr_10


def from_multi_df_result_get_tau(rdf_dict, line, isotope):
    """
    Optical depth of a transition of an isotope, specified by 'line' and 'isotope' arguments
    :param rdf_dict: dict of rdfs with isotope keys
    :param line: int 10 or 32
    :param isotope: string indicating isotope. 'co', '13co', 'c18o'
    :returns: float optical depth for that line
    """
    return from_multi_df_result_get_col(rdf_dict, line, isotope, "tau")


# Function to get grid index from cdmol and n; copied from above
def find_grid_index(coldensh2, n, cdarr, narr):
    """
    Return grid index (i, j) for this coldensh2 and n value
    The arrays from which they are taken must be given as arguments.
    Will return the FIRST instance of each. However, the arrays should only have one instance of each.
    :param coldensh2: float column density
    :param n: float density
    :param cdarr: array of column densities
    :param narr: array of densities
    :returns: tuple(i, j) where i, j are indices. This tuple can directly index a 2d array
    """
    return (np.where(cdarr == coldensh2)[0][0], np.where(narr == n)[0][0])

In [ ]:
# Global setup
tk = 30. # K

params = radex.get_default_parameters()
linewidth = 1. # km/s
# Setup fixed params
params = radex.get_default_parameters()
params['tkin'] = tk
params['fmin'] = 100.
params['fmax'] = 400.
params['linewidth'] = linewidth

# Define variable params
coldensh2_arr = 10.**np.arange(19, 25, 0.25) # 0.5 spacing
n_arr = 10.**np.arange(1, 10, 0.25) # 0.5 spacing
# Chain them together in an iterable
coldensh2_n_tup_arr = tuple(itertools.product(coldensh2_arr, n_arr))

# Result dict; figure out how to put it into arrays later (without rerunning radex a million times while debugging)
# result_dict = {}


def task_run_radex(args):
    """
    Parallel-able task (i.e. for use in Pool.map, or regular serial map)
    :param args: tuple(float coldensh2, float n)
    :returns: tuple(float coldensh2, float n, Dataframe result)
    """
    coldensh2, n = args
    params_copy = params.copy()
    params_copy['h2'] = n
    interim_result_dict = {}
    for isotope in ['co', '13co', 'c18o']:
        params_copy['molfile'] = isotope + '.dat'
        params_copy['cdmol'] = convert_nh2_to_co_isotope(coldensh2, isotope)
        interim_result_dict[isotope] = radex.run(params_copy)
    # Get both "thin" line ratios
    line_ratio_list = [from_multi_df_result_get_line_ratio(interim_result_dict, isotope10) for isotope10 in ('13co', 'c18o')]
    # Get all TRs; 5 total (no c18o32 obvi)
    # Get all taus; 5 total
    tr_list = []
    tau_list = []
    # The loop will produce the order [co10, co32, 13co10, 13co32, c18o10]
    for isotope in ['co', '13co', 'c18o']:
        for line in [10, 32]:
            if ('18' in isotope) and (line == 32):
                # No c18o 3-2 line, we don't have it.
                continue
            tr_list.append(from_multi_df_result_get_tr(interim_result_dict, line, isotope))
            tau_list.append(from_multi_df_result_get_tau(interim_result_dict, line, isotope))
    # Output value ordering:
    # [ratio_32, ratio_10, ... TRs in order ... taus in order ...]
    # See comments above for the internal order of TR and tau
    output_values = line_ratio_list + tr_list + tau_list
    # Return the input parameters (for organizing the results) and, carefully ordered?, the output values
    return coldensh2, n, output_values

# Start timer
t0 = time.perf_counter()

parallel = True
if parallel:
    # use Pool with 4 processes, which is how many CPU cores my laptop has
    with Pool(4) as pool:
        entire_result_list = pool.map(task_run_radex, coldensh2_n_tup_arr)
else:
    # serial test case
    # use list comphrehension, the more "pythonic" alternative to map()
    entire_result_list = [task_run_radex(x) for x in coldensh2_n_tup_arr]

# Mark time at end of the Radex runs
t1 = time.perf_counter()

# Compose a list of the array names in the order that the values were saved
ratio_names = ['ratio_13co32_13co10', 'ratio_13co32_c18o10']
tr_names, tau_names = [], []
for isotope in ['co', '13co', 'c18o']:
    for line in [10, 32]:
        if ('18' in isotope) and (line == 32):
            continue
        tr_names.append("TR_" + isotope + str(line))
        tau_names.append("tau_" + isotope + str(line))
output_array_names = ratio_names + tr_names + tau_names
# Create a list of empty arrays for each of the unique output values
output_arrays = [np.zeros((coldensh2_arr.size, n_arr.size)) for _ in output_array_names]


for each_result in entire_result_list:
    # Unpack result
    cdh2, n, outputs = each_result
    # Find the array index of this result
    arr_idx = find_grid_index(cdh2, n, coldensh2_arr, n_arr)
    # Unpack each of the output values into the correct array
    for i, value in enumerate(outputs):
        output_arrays[i][arr_idx] = value

# Mark time at end of output organization
t2 = time.perf_counter()
print(f"Radex runs took {t1-t0:0.4f} seconds {'using' if parallel else 'without'} a pool")
print(f"Output organization took {t2-t1:0.4f} seconds")
print(f"Total time {t2-t0:0.4f} seconds")
print(f"Grid shape {output_arrays[0].shape} and size {output_arrays[0].size}")

In [ ]:
print(params)

In [ ]:
%matplotlib nbagg
def calculate_extent(arr):
    """
    Calculate the image "extent" needed to let the "0" pixel in the image be defined as the beginning of the array
    (Deal with the 0.5s and all that)
    :param arr: array whose first element should lie at the "0" point on the image axis
    """
    diff = np.diff(arr)[0]
    tmp_arr = arr - diff/2
    return [tmp_arr[0], tmp_arr[-1]+diff]

xaxis, yaxis = np.log10(n_arr), np.log10(coldensh2_arr)
extent = calculate_extent(xaxis) + calculate_extent(yaxis)

fig = plt.figure(figsize=(9, 10))
for i, grid in enumerate(output_arrays):
    array_name = output_array_names[i]
    vlims = {}
    if 'tau' in array_name:
        grid = np.log10(grid)
        vlims['vmin'] = -6
        vlims['vmax'] = 5
    plt.subplot(4, 3, 1+i)
    plt.imshow(grid,
               extent=extent,
    #            interpolation='nearest',
              origin='lower', **vlims)
    plt.colorbar(label=array_name, orientation='horizontal')
    plt.xlabel("log10 n")
    plt.ylabel("log10 N(H2)")
plt.tight_layout()
parallel_stub = "parallel" if parallel else "serial"
size_stub = f"{output_arrays[0].size:04d}"
plt.savefig(f"/home/ramsey/Pictures/2023-12-13/co_radex_grid_size{size_stub}_{parallel_stub}.png")

In [ ]:
# Save these to a FITS file
hdu_list = [fits.PrimaryHDU()]

header_template = fits.Header({
    'CREATOR': "Ramsey Karim via co_column_grid.ipynb",
    'DATE': f"Created: {datetime.datetime.now(datetime.timezone.utc).astimezone().isoformat()}",
    'COMMENT': f"Kinetic temperature fixed at {tk:.1f} K",
    'COMMENT': "Grids created using the spectralradex wrapper of Radex",
})

save_path = "/home/ramsey/Documents/Research/Feedback/misc_data/co_grids"
save_name = f"co_radex_grid_Tk{tk:.1f}K_size{size_stub}.fits"
save_full_name = os.path.join(save_path, save_name)


cd_grid, n_grid = np.meshgrid(coldensh2_arr, n_arr, indexing='ij')
all_array_names = ["NH2", "n"] + output_array_names
all_arrays = [cd_grid, n_grid] + output_arrays

def decide_units(name):
    easy_ones = {"NH2": str(u.cm**-2), 'n': str(u.cm**-3)}
    if name in easy_ones:
        return easy_ones[name]
    elif 'ratio' in name:
        return ''
    elif 'TR' in name:
        return 'K'
    elif 'tau' in name:
        return ''

for i, grid in enumerate(all_arrays):
    array_name = all_array_names[i]
    hdr = header_template.copy()
    hdr['BUNIT'] = decide_units(array_name)
    hdr['EXTNAME'] = array_name
    hdu_list.append(fits.ImageHDU(data=grid, header=hdr))

hdul = fits.HDUList(hdu_list)
hdul.writeto(save_full_name)
